<a href="https://colab.research.google.com/github/lainey-reed/Textbook/blob/master/Colab/AC_Textbook_Chapter8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 8: Flocculation

All of the code examples in this colab file are from [Chapter 8: Flocculation](https://aguaclara.github.io/Textbook/Flocculation/Floc_Intro.html)

In [ ]:
!pip install aguaclara

## Flocculation Introduction

The following examples are from the chapter section [Flocculation Introduction](https://aguaclara.github.io/Textbook/Flocculation/Floc_Intro.html#)

### Example 1: Number of collisions in a floc

The example below is from the subsection [Number of Particles in a Floc](https://aguaclara.github.io/Textbook/Flocculation/Floc_Intro.html#number-of-particles-in-a-floc)

In [ ]:
import numpy as np

n_primary = 1000000000
n_collisions = np.log10(n_primary)/np.log10(2)
print(n_collisions)

### Example 2: Collision Potential

The example below is from the subsection [Flocculation is (or used to be) a Slow Process](https://aguaclara.github.io/Textbook/Flocculation/Floc_Intro.html#flocculation-is-or-used-to-be-a-slow-process)

In [ ]:
import aguaclara as ac
from aguaclara.core.units import unit_registry as u
import numpy as np

HL_floc = 43*u.cm
HRT = 8 * u.min
Temperature =20 * u.degC
G_floc = ((u.gravity*HL_floc/(HRT*ac.viscosity_kinematic_water(Temperature)))**0.5).to_base_units()
print(G_floc)
Gt_floc = G_floc*HRT
HRT_floc_visible = 0.5*u.min
Gt_floc_visible = (G_floc*HRT_floc_visible).to_base_units()
print(Gt_floc_visible)

## Flocculation Model

The next three examples are from the [Flocculation Model](https://aguaclara.github.io/Textbook/Flocculation/Floc_Model.html#flocculation-model) section, in the subsection [Key understanding: Relative velocities between particles are dominated by viscous shear](https://aguaclara.github.io/Textbook/Flocculation/Floc_Model.html#key-understanding-relative-velocities-between-particles-are-dominated-by-viscous-shear)

In [ ]:
import aguaclara
import aguaclara.core.physchem as pc
from aguaclara.core.units import unit_registry as u
import aguaclara.core.constants as con
import aguaclara.research.environmental_processes_analysis as epa
import aguaclara.research.floc_model as fm

import numpy as np
import matplotlib.pyplot as plt

C_Clay = np.arange(1,1000,1)*u.NTU
n_Clay = fm.particle_number_concentration(C_Clay,fm.Clay)
fig, ax = plt.subplots()
ax.loglog(C_Clay.to(u.NTU),n_Clay.to(1/u.L))
ax.set(xlabel='Clay concentration ($NTU$)', ylabel='Number of clay per liter')
#fig.savefig('Flocculation/Images/NClay_vs_CClay')
plt.show()


In [ ]:
import aguaclara.core.physchem as pc
from aguaclara.core.units import unit_registry as u
import aguaclara.core.constants as con
import aguaclara.research.environmental_processes_analysis as epa
import aguaclara.research.floc_model as fm

import numpy as np
import matplotlib.pyplot as plt
lamda_Clay = fm.sep_dist_clay(C_Clay,fm.Clay)
fig, ax = plt.subplots()
ax.semilogx(C_Clay.to(u.NTU),lamda_Clay.to(u.mm))
ax.set(xlabel='Clay concentration ($NTU$)', ylabel=r'Clay separation distance ($mm$)')
#fig.savefig('Flocculation/Images/LambdaClay_vs_CClay')
plt.show()

In [ ]:
import aguaclara.core.physchem as pc
from aguaclara.core.units import unit_registry as u
import aguaclara.core.constants as con
import aguaclara.research.environmental_processes_analysis as epa
import aguaclara.research.floc_model as fm

import numpy as np
import matplotlib.pyplot as plt
Temperature = 20 * u.degC
G=np.arange(1,1000,1)*u.Hz
EDR = G**2 * pc.viscosity_kinematic_water(Temperature)
Inner_viscous = fm.lambda_vel(EDR, Temperature)
fig, ax = plt.subplots()
ax.semilogx(G.to(u.Hz),Inner_viscous.to(u.mm))
ax.set(xlabel='Velocity gradient (Hz)', ylabel='Inner viscous length scale (mm)')
ax.text(10, 17, 'Eddies cause mixing', fontsize=12,rotation=-30)
ax.text(3, 7, 'Viscous shear', fontsize=12,rotation=-30)
#fig.savefig('Flocculation/Images/innerviscous_vs_G')
plt.show()

## Flocculation Examples

The following code is from [Flocculation Examples](https://aguaclara.github.io/Textbook/Flocculation/Floc_Examples.html)

(It does throw errors but the notes at the beginning of the chapter say it is going to be reworked)

In [ ]:
from aguaclara.core import pipes
# %%
#Assumptions
Pi_VC = .62 #Vena contracta coefficient of an orifice
Ke = ((1/Pi_VC**2)-1)**2 #expansion coefficient

#Functions to calculate key parameters

def Gave(G_theta,h_floc,Temp):
    """Calculates average G given target minimum collision potential, total headloss, and design temperature
    equation from flocculation slides"""
    G_ave = (u.gravity*h_floc/(G_theta*pc.viscosity_kinematic_water(Temp))).to(1/u.s)
    return G_ave

def restime(G_theta,G_ave):
    """Calculates residence time given collision potential and average G
    equation from flocculation slides"""
    theta = G_theta/G_ave
    return theta


def Dpipe(Ke,Pi_HS,Q,G_ave,Temp,SDR):
    """Calculates the actual inner diameter of the pipe
    equation from flocculation slides"""
    D_pipe = ((Ke/(2*Pi_HS*pc.viscosity_kinematic(Temp)*G_ave**2))*(4*Q.to(u.m**3/u.s)/np.pi)**3)**(1/7)
    return D_pipe

def Keactual(ID_pipe,G_ave,Temp,Pi_HS,Q):
    """estimates actual expansion coefficient given the actual inner diameter and other relevant inputs
    equation from flocculation slides"""
    Ke_actual = np.pi**3*ID_pipe**7*G_ave**2*pc.viscosity_kinematic(Temp)*Pi_HS/(32*Q.to(u.m**3/u.s)**3)
    return Ke_actual



def Aorifice(ID_pipe,Ke_actual,Temp,Q):
    """Calculates the orifice area given pipe inner diameter, expansion coefficient, Temperature, and flow"""
    A1 = (pc.area_circle(ID_pipe)).to(u.cm**2).magnitude #Pipe area
    Nu = pc.viscosity_kinematic(Temp) #kinematic viscocity
    Re = pc.re_pipe(Q,ID_pipe,Nu) #reynolds number

    def f_orif(A2,A1,Ke_actual,Re): #root of this function is the orifice area
        return (2.72+(A2/A1)*(4000/Re))*(1-A2/A1)*((A1/A2)**2-1)-Ke_actual

    A_orifice = (brentq(lambda A2: f_orif(A2,A1,Ke_actual,Re), -1, 2*A1))*u.cm**2 #numerical optimization

    return A_orifice


def eave(G_ave,Temp):
    """Calculates the average energy dissipation rate"""
    e_ave = (pc.viscosity_kinematic(Temp)*G_ave**2).to(u.mW/u.kg)
    return e_ave

def Hchip(A_orifice,ID_pipe):
    """This function calculates the height of the chip based on the orifice area and pipe diameter
    The function uses numerical optimization to solve the transcendental equation"""
    A_flow = A_orifice.magnitude #orifice area stripped of units
    r=(ID_pipe/2).magnitude #radius stripped of units
    c = A_flow/r**2 #left hand side of equation

    def f(a,c): #roots of this function are theta
        return a-sin(a)*cos(a)-c

    theta = brentq(lambda a: f(a,c), 0, 13) #numerical optimization
    r_u = r*u.cm #radius with units
    y = r_u - r_u*np.cos(theta) #height of orifice

    H_chip = ID_pipe-y #height of chip
    return H_chip

def Cost_Length(L_pipe,ND_pipe):
    """This function calculates the total cost of the system and the total length of the system"""
    #Length of pipe and number of fittings needed
    OD_pipe = pipe.OD(ND_pipe)
    Total_Pipe = L_pipe + .5*u.m
    Number_T = np.ceil(Total_Pipe.magnitude)
    Number_Elbow = np.ceil(Total_Pipe.magnitude)

    if ND_pipe.magnitude == 3:
        Cost_T = 3.94*u.dollar
        Cost_Elbow = 3.53*u.dollar
        Cost_Pipe = (17.14/10*(u.dollar/u.foot)).to(u.dollar/u.m)
        Cost_Valve = 10*u.dollar
        Width_T = (3.99*u.inch).to(u.cm)
        Width_Elbow = (3.97*u.inch).to(u.cm)


    if ND_pipe.magnitude ==4:
        Cost_T = 7.16*u.dollar
        Cost_Elbow = 5.40*u.dollar
        Cost_Pipe = (21.5/10*(u.dollar/u.foot)).to(u.dollar/u.m)
        Cost_Valve = 10*u.dollar
        Width_T = (5.06*u.inch).to(u.cm)
        Width_Elbow = (5.06*u.inch).to(u.cm)

    if ND_pipe.magnitude ==6:
        Cost_T = 7.16*u.dollar
        Cost_Elbow = 5.40*u.dollar
        Cost_Pipe = (21.5/10*(u.dollar/u.foot)).to(u.dollar/u.m)
        Cost_Valve = 10*u.dollar
        Width_T = (5.06*u.inch).to(u.cm)
        Width_Elbow = (5.06*u.inch).to(u.cm)


    Total_Cost = Cost_Pipe*Total_Pipe + Cost_T*Number_T + Cost_Elbow*Number_Elbow + Cost_Valve*Number_Elbow
    Floor_Length = Number_T*(Width_T+Width_Elbow-OD_pipe).to(u.m)
    Output=[Total_Cost,Floor_Length]
    return Output
#Inputs
D_Sed = 2.5*u.cm
A_Sed = pc.area_circle(D_Sed)
v_Sed = 2*u.mm/u.s
Q = (v_Sed*A_Sed).to(u.mL/u.s)
print('The flow rate is',Q)

Temp = 15*u.degC
h_floc = 50*u.cm #standard for Aguaclara plants
G_theta = 20000 #standard for Aguaclara plants
Pi_HS = 6  ##3-6 is a good range, more research needed
SDR = 41 #Standard ratio
#Calculate G average using functions listed above and given inputs
G_ave = Gave(G_theta,h_floc,Temp)
theta = restime(G_theta,G_ave)
e_ave = eave(G_ave,Temp)
print('The average G value is ',G_ave)
print('The residence time in the flocculator is ',theta)
print('The average energy dissipation rate is ', e_ave)
#Calculate the pipe diameter, both inner and nominal and determine area of pipe using inner diameter output
D_pipe = (Dpipe(Ke,Pi_HS,Q,G_ave,Temp,SDR)).to(u.cm)
#Calculate nominal diameter of pipe
ND_pipe = pipes.ND_SDR_available(D_pipe,SDR)
#Calculate nominal diameter of pipe
ID_pipe = pipes.ID_SDR(ND_pipe,SDR).to(u.cm)

ID_pipe = 5*u.mm
#Calculate inner diameter of pipe
A_pipe = (pc.area_circle(ID_pipe)).to(u.cm**2)

print('The ideal inner diameter of the pipe would be ',D_pipe)
print('The nominal diameter of the pipe is ',ND_pipe, ', and the inner diameter is ', ID_pipe)
print('The area of the pipe is ', A_pipe)
#Calculate the actual Ke as a result of the calculated inner pipe diameter
Ke_actual = (Keactual(ID_pipe,G_ave,Temp,Pi_HS,Q)).to(u.dimensionless)
print('The initial expansion minor loss coefficient was ',Ke)
print('The actual expansion minor loss coefficient is ',Ke_actual)
#Calculate the orifice area
A_orifice = Aorifice(ID_pipe,Ke_actual,Temp,Q)
print('The orifice area is ',A_orifice)
# The following line of code needs to be removed once the orifice area equation is corrected.

H_chip = Hchip(A_orifice,ID_pipe)
print('The height of the chip is ', H_chip)
#Calculate average velocity
v_avg = (Q/pc.area_circle(ID_pipe)).to(u.m/u.s) #first calculate average velocity
print('The average velocity is ',v_avg)

#Calculate pipe length
L_pipe = (v_avg*theta).to(u.m) #then multiply velocity by residence time to get the required length of pipe
print('The length of the pipe is ',L_pipe)

## Mechanical Flocculator Solution

The following snippets of code are from the section [Mechanical Flocculator Solution: Mechanical Flocculator Design](https://aguaclara.github.io/Textbook/Flocculation/Floc_Mechanical_Solution.html#mechanical-flocculator-design)

In [ ]:
import aguaclara.core.physchem as pc
from aguaclara.core.units import unit_registry as u

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
flow_plant = 50 * u.L/u.s
G_mech = 70 / u.s
temp_design = 10 * u.degC

In [ ]:
#answer
time_mech = 30 * u.min
Gtime_mech= (G_mech * time_mech).to(u.dimensionless)

print('The Gt is', Gtime_mech)

In [ ]:
#answer
ed_rate_mech_ave = (G_mech**2 * pc.viscosity_kinematic_water(temp_design)).to(u.mW/u.kg)
print('The equivalent average energy dissipation rate is' , ed_rate_mech_ave, '.')

In [ ]:
#answer
def power_floc_shaft(Q, G, t, temp):
    return (G**2 * Q * t * pc.viscosity_dynamic_water(temp)).to(u.kW)
power_mech_floc = power_floc_shaft(flow_plant,G_mech,time_mech,temp_design)
print('The power requirement is', power_mech_floc,'.')

In [ ]:
#answer
ratio_prop_vel = 0.75
pi_plate = 0.04
vel_prop = 3 * u.ft/u.s
height_prop = 3 * u.cm

ed_rate_prop_max = pi_plate * ((ratio_prop_vel *  vel_prop)**3 / height_prop).to(u.mW/u.kg)

print('The maximum energy dissipation rate behind the propeller tip is', ed_rate_prop_max)

In [ ]:
#answer
ed_rate_mech_ratio = ed_rate_prop_max / ed_rate_mech_ave
print('The ratio of maximum to average energy dissipation rate is', ed_rate_mech_ratio,'.')

G_mech_ratio = ed_rate_mech_ratio**0.5
print('The ratio of maximum to average velocity gradient is', G_mech_ratio,'.')

# Answers for these are slightly off from answers in textbook

In [ ]:
#answer
G_hyd_ratio = np.sqrt(2)
G_mech_ave_max = 180 * u.Hz
G_hyd_ave_max = G_mech_ave_max * (G_mech_ratio/G_hyd_ratio)
print('The maximum G for hydraulic flocculators is', G_hyd_ave_max, '.')

In [ ]:
#answer
def energy_mech(Q, G, t, temp):
    return (power_floc_shaft(Q, G, t, temp) / Q).to(u.J/u.L)


print('The energy required using the mechanical flocculator is', energy_mech(flow_plant,G_mech,time_mech,temp_design),'.')

In [ ]:
#answer
efficiency_motor = 0.8
electricity_rate = ((0.15 * u.USD) / (u.kW * u.hr))
electricity_cost_mech = (electricity_rate * energy_mech(flow_plant,G_mech,time_mech,temp_design) / efficiency_motor).to(u.USD/u.ML)
print('The cost of electricity for mechanical flocculation is', electricity_cost_mech,'.')

In [ ]:
#answer
delta_height = (power_floc_shaft(flow_plant,G_mech,time_mech,temp_design) / (flow_plant * pc.density_water(temp_design) * u.gravity)).to(u.m)

print('The equivalent amount of potential energy to run this  mechanical flocculator is', delta_height,'.')

print('The shaft power required for this flocculator is ', power_floc_shaft(flow_plant,G_mech,time_mech,temp_design),'.')

In [ ]:
#answer
vol_mech = (time_mech * flow_plant).to(u.m**3)
print('The required reactor volume for the mechanical flocculator is', vol_mech,'.')

In [ ]:
#answer
depth_mech = 4 * u.m
area_mech = (vol_mech / (depth_mech * flow_plant))
print('The required plan view area is', area_mech)

## Flocculation Model Solution

The following code is from the [Flocculation Model Solution](https://aguaclara.github.io/Textbook/Flocculation/Floc_Model_Solution.html) section.

Certain functions were removed from the aguaclara package as the underlying equations are under suspicison. As a result, this code cannot be run in its entirety and sections have been commented out.

In [ ]:
import aguaclara.core.physchem as pc
from aguaclara.core.units import unit_registry as u

import aguaclara.research.floc_model as fm

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ratio_prop_vel = 0.75
pi_plate = 0.04
vel_prop = 3 * u.ft/u.s
height_prop = 3 * u.cm

ed_rate_prop_max = pi_plate * ((ratio_prop_vel *  vel_prop)**3 / height_prop).to(u.mW/u.kg)

#code is removed due to removal of diam_flox_m
#answer
#diam_floc_mech = fm.diam_floc_max(ed_rate_prop_max).to(u.um)
#print('The diameter of the flocs that interact with the impeller is', diam_floc_mech, '.')
#x=fm.DIM_FRACTAL
#x

In [ ]:
#answer
conc_Al = 1.5 * u.mg/u.L
conc_clay=100*u.NTU
#vel_term_floc_tip = fm.vel_term_floc(conc_Al, conc_clay, fm.PACl,
#                                      fm.Clay, fm.DIM_FRACTAL,
#                                      diam_floc_mech, temp_design).to(u.mm/u.s)

#print('The terminal velocity of flocs that interact with the impeller tip is estimated to be', vel_term_floc_tip)

In [ ]:
#answer
vel_capture_10_state = (1.2 * u.m/u.hr).to(u.mm/u.s)
print('The 10 State Standards capture velocity is', vel_capture_10_state)
print("The 10 State Standards sedimentation tank would capture the flocs that are able to survive the energy dissipation rate at the tip of the propeller. ")

In [ ]:
#answer
init_sep_dist_clay = fm.sep_dist_clay(100 * u.NTU, fm.Clay).to(u.mm)
final_sep_dist_clay = fm.sep_dist_clay(1 * u.NTU, fm.Clay).to(u.mm)
print('The average distance between clay particles at 100 NTU is', init_sep_dist_clay)
print('The average distance between clay particles at 1 NTU is', final_sep_dist_clay)

In [ ]:
#answer
print('The inner viscous length scale is', fm.lambda_vel(ed_rate_prop_max, temp_design).to(u.mm))

In [ ]:
#This code is provided to help you make your graph
fig, ax = plt.subplots()

#Creates the array for energy dissipation rates (EDRs)
x = np.logspace(np.log10(1),4)*u.mW/u.kg

ax.text(10, 2, 'Inner Viscous Scale', fontsize=12,rotation=-30)
ax.set(title='Inner Viscous Scale vs Energy Dissipation Rate')

ax.set(ylabel='Inner Viscous Scale (mm)')
ax.set(xlabel='Energy Dissipation Rate(mW/kg)')

#plt.grid(b=True, which='major', color='k', linestyle='-', linewidth=1)
#plt.grid(b=True, which='minor', color='k', linestyle='-', linewidth=0.5)

#fm.lambda_vel, which returns the inner viscous length scale,
#is being applied to the array of EDRs for our design temperature
y = fm.lambda_vel(x,temp_design)
ax.loglog(x, y.to(u.mm))

#------------------------------------------------------------------------
#----------------------------YOUR CODE BELOW-----------------------------
#------------------------------------------------------------------------
#answer
ax.plot(ed_rate_prop_max.to(u.mW/u.kg), final_sep_dist_clay.to(u.mm), 'ko')


#print(x)
#print(y)
print(final_sep_dist_clay)
print(ed_rate_prop_max)

plt.show()

In [ ]:
diam_tube = 7.5 * u.cm
conc_clay = 100*u.NTU
conc_Al = 0.5*u.mg/u.L


#fm.ratio_area_clay_total() returns fraction (between 0 and 1) that represents the surface area of
#the clay particle over the sum of the surface area of the clay and reactor walls

wall_loss = 1 - fm.ratio_area_clay_total(conc_clay, fm.Clay, diam_tube, fm.RATIO_HEIGHT_DIAM)
print('The fraction of the coagulant lost to the walls is', wall_loss)

#fm.gamma_coag() returns the fraction of clay that is covered by coagulant. This is a very hard parameter
#to actually measure, so this is just an estimate.

fraction_coated = fm.gamma_coag(conc_clay,conc_Al, fm.PACl,
                                  fm.Clay, diam_tube, fm.RATIO_HEIGHT_DIAM)
print('The fraction of the clay surface area that is is coated is', fraction_coated)

In [ ]:
ed_rate_mech_ave = 6.4*u.mW/u.kg

time_first_collision = fm.time_col_laminar(ed_rate_mech_ave, 10*u.degC,
                                             conc_Al, conc_clay, fm.PACl,
                                             fm.Clay, fm.Clay.Diameter, diam_tube,
                                             fm.DIM_FRACTAL, fm.RATIO_HEIGHT_DIAM).to(u.s)

print('The time required for the first succesful collision is', time_first_collision)

In [ ]:
#answer
ed_rate_floc_aguaclara = 11*u.mW/u.kg
time_floc_aguaclara = 8.1*u.minute
temp_design_aguaclara = 15*u.degC

#This equation for G can be found in the course slides (all equations you see in design challenges can be found in the slides)
G_floc_aguaclara = np.sqrt(ed_rate_floc_aguaclara/pc.viscosity_kinematic_water(temp_design_aguaclara))
Gtime_floc_aguaclara = (G_floc_aguaclara*time_floc_aguaclara).to(u.dimensionless)
print('The AguaClara Gt value is', Gtime_floc_aguaclara)

In [ ]:
#Fitting constant/Sedimentation tank factor
k = 0.24

#gamma_aguaclara_design uses functions in floc_model.py to solve the equation in the problem statement
conc_clay_goal = 2 * u.NTU
gamma_aguaclara_design = ((3/2) * (fm.sep_dist_clay(conc_clay_goal, fm.Clay)**2
                           / (k * np.pi * (fm.Clay.Diameter * u.m)**2
                              * Gtime_floc_aguaclara
                             )
                          )).magnitude
#gamma_aguaclara_design=gamma_aguaclara_design.magnitude

print('The Gamma value is', gamma_aguaclara_design)

In [ ]:
fig, ax = plt.subplots()
#Define the range of coagulant. This is necessary to create plots of pC* as a function of coagulant dose.
coag_graph = np.linspace(0.01, 2.5, 100) * u.mg/u.L

# Graph results of a particular NTU. Note that you can change this value to see how the graph responds.
# A change here even changes the graph title changes!
plot_NTU = 50*u.NTU

#plt.figure(str(plot_NTU), (6,6))
ax.set(title=(str(plot_NTU)+' Graph for Various Humic Acid Concentrations'))
ax.set(ylabel='pC*')
ax.set(xlabel='coagulant dosage (mg/L)')

# Create an array of humic acid concentrations
plot_humic_acid = np.linspace(0,15,6)*u.mg/u.L

# Create a function that only has inputs for the values that we will change between plots.
#All other variables are taken from predefined values. This simplifies the function call for use in generating the plots.
def plot_pC(conc_humic_acid):
    k = 0.24
    # The energy dissipation rate for aguaclara designs
    ed_rate = 11*u.mW/u.kg
    #The inner diameter of the flocculator tube is important because a significant fraction of the
    #coagulant ends up attaching to the flocculator walls
    tube_diam = 3/8 * u.inch
    time_floc = 8.1 * u.minute
    temp = 15 * u.degC
    #pc_viscous is the solution for the equation in the problem statement. It returns pC*
    plot_pC = fm.pc_viscous(ed_rate, temp, time_floc, tube_diam,
                            plot_NTU, coag_graph, conc_humic_acid,
                            fm.HumicAcid, fm.PACl, fm.Clay,
                            k, fm.RATIO_HEIGHT_DIAM)
    return plot_pC

x = coag_graph.to(u.mg/u.L)

ax.plot(x, plot_pC(plot_humic_acid[0]), 'r',
         x, plot_pC(plot_humic_acid[1]), 'b',
         x, plot_pC(plot_humic_acid[2]), 'g',
         x, plot_pC(plot_humic_acid[3]), 'm',
         x, plot_pC(plot_humic_acid[4]), 'c',
         x, plot_pC(plot_humic_acid[5]), 'y')

#We can use the array of humic acid concentrations to directly create the legend!
plt.legend(plot_humic_acid, loc = 'best')
plt.show()

In [ ]:
fig, ax = plt.subplots()

#answer
plot_NTU = 10*u.NTU

#plt.figure(str(plot_NTU), (6,6))
ax.set(title=(str(plot_NTU)+' Graph'))
ax.set(ylabel='Settled water turbidity (NTU)')
ax.set(xlabel='coagulant dosage (mg/L)')

def plot_conc_clay(conc_clay, conc_nat_org_mat):
    k = 0.24
    ed_rate = 11*u.mW/u.kg
    #The inner diameter of the flocculator tube is important because a significant fraction of the
    #coagulant ends up attaching to the flocculator walls
    tube_diam = 3/8 * u.inch
    conc_clay = plot_NTU
    time_floc = 8.1 * u.minute
    temp = 15 * u.degC
    #s_t calls on fm.pc_viscous() like the previous cell of code, but also uses the fm.invp()
    #function to turn pC* back into units of settled water turbididty.
    s_t = fm.invp(fm.pc_viscous(ed_rate, temp, time_floc, tube_diam,
               conc_clay, coag_graph, conc_nat_org_mat, fm.HumicAcid, fm.PACl, fm.Clay,
                            k, fm.RATIO_HEIGHT_DIAM),conc_clay)

    return s_t

#Creates array of humic acid concentrations. This was done in the previous code cell with np.linspace,
#but because we don't want evenly spaced concentrations here we input our desired values manually
plot_humic_acid = np.array([0,1,5,20])*u.mg/u.L

x = coag_graph.to(u.mg/u.L)
ax.plot(x, plot_conc_clay(plot_NTU, plot_humic_acid[0]), 'r',
         x, plot_conc_clay(plot_NTU, plot_humic_acid[1]), 'b',
         x, plot_conc_clay(plot_NTU, plot_humic_acid[2]), 'g',
         x, plot_conc_clay(plot_NTU, plot_humic_acid[3]), 'y')

plt.legend(plot_humic_acid, loc = 'best')
plt.show()

## Flocculator Design Solution

The following code is from the final section of this chapter [Flocculator Design Solution](https://aguaclara.github.io/Textbook/Flocculation/Floc_Design_Solution.html)

In [ ]:
#import statements have been cleaned up
import aguaclara.core.physchem as pc
from aguaclara.core.units import unit_registry as u
import aguaclara.core.constants as constants
import numpy as np
import matplotlib.pyplot as plt
#plt.switch_backend('TKAgg')

In [ ]:
# head loss through the flocculator
headloss_floc_BOD = 40 * u.cm

# collision potential based on recent designs for AguaClara Plants
Gt_BOD = 37000

# water depth at the end of flocculator where it flows into the inlet channel of the sedimentation tank
height_floc_end = 2 * u.m

flow_plant = 20 * u.L/u.s

# This is the estimate for larger plants where the flocculator is as long as the sedimentation tanks.
# For lower flow plants we will need to reduce this length because of the constraint that the channels must
# be wide enough to construct.
length_channel_max = 6 * u.m

# minimum and maximum ratios of distance between expansions to baffle spacing
Pi_HS_min = 3
Pi_HS_max = 6

Pi_vc = constants.VC_ORIFICE_RATIO
Pi_vc_baffle = Pi_vc**2

# width of the polycarbonate sheets used to make baffles
width_PC_sheet = 1.067*u.m

# this is a reasonable constraint to keep the channel constructable by humans
width_floc_min_BOD = width_PC_sheet / 2
width_floc_max_BOD = width_PC_sheet

# expansion minor loss coefficient for 180 degree bend
K_e = (1 / Pi_vc_baffle - 1)**2

# this is the minimum temperature of the raw water
T_BOD = 15* u.degC

### Hydraulic Vertical Flow Flocculator Design
The code can be found [here](https://aguaclara.github.io/Textbook/Flocculation/Floc_Design_Solution.html#hydraulic-vertical-flow-flocculator-design)

In [ ]:
# head loss through the flocculator
headloss_floc_BOD = 40 * u.cm

# collision potential based on recent designs for AguaClara Plants
Gt_BOD = 37000

# water depth at the end of flocculator where it flows into the inlet channel of the sedimentation tank
height_floc_end = 2 * u.m

flow_plant = 20 * u.L/u.s

# This is the estimate for larger plants where the flocculator is as long as the sedimentation tanks.
# For lower flow plants we will need to reduce this length because of the constraint that the channels must
# be wide enough to construct.
length_channel_max = 6 * u.m

# minimum and maximum ratios of distance between expansions to baffle spacing
Pi_HS_min = 3
Pi_HS_max = 6

Pi_vc = constants.VC_ORIFICE_RATIO
Pi_vc_baffle = Pi_vc**2

# width of the polycarbonate sheets used to make baffles
width_PC_sheet = 1.067*u.m

# this is a reasonable constraint to keep the channel constructable by humans
width_floc_min_BOD = width_PC_sheet / 2
width_floc_max_BOD = width_PC_sheet

# expansion minor loss coefficient for 180 degree bend
K_e = (1 / Pi_vc_baffle - 1)**2

# this is the minimum temperature of the raw water
T_BOD = 15* u.degC

In [ ]:
def G_avg(hl, Gt, T):
    G = (u.gravity * hl) / (Gt * pc.viscosity_kinematic_water(T_BOD))
    return G.to(1/u.s)

print ('The average velocity gradient of flocculator is', G_avg(headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
def t_floc_est(hl, Gt, T):
    theta = Gt / G_avg(hl, Gt, T)
    return theta.to(u.s)

print ('The residence time of flocculator is', t_floc_est(headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
def Gt_operation(hl,Gt,T_BOD,T):
    Gt_operation = np.sqrt(u.gravity * hl * t_floc_est(hl, Gt, T_BOD)/pc.viscosity_kinematic_water(T))
    return Gt_operation.to(u.dimensionless)
Temp_Operation=np.linspace(0, 30)*u.degC
ypoints=(Gt_operation(headloss_floc_BOD, Gt_BOD, T_BOD,Temp_Operation))
fig, ax = plt.subplots()
ax.plot(Temp_Operation,ypoints,'-')
#Note the use of latex. The r tells matplotlib to interpret the following strings as raw text.
#Without the "r" the \theta would be parsed as a tab (\t) followed by heta!
ax.set(xlabel=r'Temperature ($^\circ$C)')
ax.set(ylabel=r'G$\theta$')
#fig.savefig('Flocculation/Images/Gtheta_vs_temperature')
plt.show()

In [ ]:
def vol_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):
    vol = t_floc_est(headloss_floc_BOD, Gt_BOD, T_BOD )*flow_plant
    return vol.to(u.m**3)

print('The volume of flocculator is', vol_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
def length_channel(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):

    #The factor of two below is because the minimum number of channels in the flocculator is two.
    #This is a result of the orientation of the sedimentation tanks and the plumbing in the plant.
    #Unless the design is for very low flows (<10 L/s), there will always be an even number of flocculator channels.
    length = vol_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD)/width_floc_min_BOD/2/height_floc_end
    return min(length,length_channel_max)

print('The maximum channel length is',length_channel(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
def width_floc_total(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):
    width_floc = vol_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD)/(length_channel(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD)*height_floc_end)
    return width_floc.to(u.m)

print ('The total width of the flocculator channels is is', width_floc_total(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
def width_floc_min_est(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):
    Gavg = G_avg(headloss_floc_BOD, Gt_BOD, T_BOD)
    nu = pc.viscosity_kinematic_water(T_BOD)
    width_floc_min = Pi_HS_min*((K_e/(2 * height_floc_end * (Gavg**2) * nu))**(1/3))*flow_plant/height_floc_end
    return width_floc_min.to(u.cm)

print('The minimum channel width is', width_floc_min_est(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
def width_floc_min(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):
    return max(width_floc_min_est(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD),width_floc_min_BOD)

print('The minimum channel width is', width_floc_min(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
def num_channel(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):
     num = (width_floc_total(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD)/
        (width_floc_min(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))).to(u.dimensionless)
     # floor function with step size 2
     num = np.floor(num/2)*2
     return int(max(num,2))

print('There are', num_channel(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD),'channels.')

In [ ]:
def width_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):
    width_total = width_floc_total(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD)
    num_c = num_channel(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD)
    return (width_total/num_c).to(u.cm)

print('The actual flocculator channel width is', width_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
def height_exp_max(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):

    g_avg = G_avg(headloss_floc_BOD, Gt_BOD, T_BOD)
    nu = pc.viscosity_kinematic_water(T_BOD)
    term1 = (K_e/(2 * (g_avg**2) * nu))**(1/4)

    term2 = (Pi_HS_max*flow_plant/width_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))**(3/4)

    height_exp = term1*term2
    return height_exp.to(u.m)

print('The maximum distance between expansions', height_exp_max(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
def num_expansions(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):
    return int(np.ceil(height_floc_end/(height_exp_max(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))).to(u.dimensionless))

print('The number of expansions is', num_expansions(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
def height_exp(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):

    return height_floc_end/num_expansions(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD)


print('The actual distance between expansions is', height_exp(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
def spacing_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):

    g_avg = G_avg(headloss_floc_BOD, Gt_BOD, T_BOD)
    nu = pc.viscosity_kinematic_water(T_BOD)
    term1 = (K_e/(2 * height_exp(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD) * (g_avg**2) * nu))**(1/3)

    ans =  term1*flow_plant/width_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD)
    return ans.to(u.m)

print ('The spacing between baffles is', spacing_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
def num_baffles(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):
    num = round(num_channel(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD)*length_channel(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD)/spacing_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))
    return int(num)

print ('The number of baffle spaces that would fit in the channels is', num_baffles(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
def Gt_baffle(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):

    term1 = spacing_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD)*width_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD)
    ans = term1*G_avg(headloss_floc_BOD, Gt_BOD, T_BOD)*height_floc_end/flow_plant
    return ans.to(u.dimensionless)

print ('The collision potential (Gt) per baffle space is', Gt_baffle(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

def num_baffle_min(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):

    ans = round(Gt_BOD/Gt_baffle(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))
    return int(ans)

print ('The minimum number of baffles required is', num_baffle_min(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
if num_baffles(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD) == num_baffle_min(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):
    print('Yes')
else:
    print('No')

In [ ]:
def vel_floc_ave(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):

    ans = flow_plant/( spacing_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD)*width_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))
    return ans.to(u.m/u.s)

print ('The average velocity of the water in the flocculator is', vel_floc_ave(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
HEIGHT_WATER_FLOC_START = height_floc_end + headloss_floc_BOD
print ('The depth of the water at the beginning of the flocculator is', HEIGHT_WATER_FLOC_START)

In [ ]:
def theta_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD):

    theta_est = t_floc_est(headloss_floc_BOD, Gt_BOD, T_BOD)
    ans = theta_est*((height_floc_end + (headloss_floc_BOD/2))/height_floc_end)
    return ans.to(u.min)

print ('The residence time in the hydraulic flocculator is', theta_floc(flow_plant, headloss_floc_BOD, Gt_BOD, T_BOD))

In [ ]:
plot_points=100
flow_plant=np.linspace(10,100, num=plot_points, endpoint=True)*u.L/u.s

expansions=np.zeros(plot_points)
for i in range(plot_points):
    expansions[i] = num_expansions(flow_plant[i], headloss_floc_BOD, Gt_BOD, T_BOD)
fig, ax = plt.subplots()
ax.plot(flow_plant,expansions,'-')
ax.set(xlabel='Plant Flow Rate (L/s)')
ax.set(ylabel='Expansion per baffle space')
#fig.savefig('Flocculation/Images/Expansions_per_baffle_space_vs_plant_flow')
plt.show()

channels=np.zeros(plot_points)
for i in range(plot_points):
    channels[i]=num_channel(flow_plant[i], headloss_floc_BOD, Gt_BOD, T_BOD)
fig, ax = plt.subplots()
ax.plot(flow_plant,channels,'-')
ax.set(xlabel='Plant Flow Rate (L/s)')
ax.set(ylabel='Number of channels')
#fig.savefig('Flocculation/Images/Number_of_channels_vs_plant_flow')
plt.show()

baffles=np.zeros(plot_points)
for i in range(plot_points):
    baffles[i]=num_baffles(flow_plant[i], headloss_floc_BOD, Gt_BOD, T_BOD)
fig, ax = plt.subplots()
ax.plot(flow_plant,baffles,'-')
ax.set(xlabel='Plant Flow Rate (L/s)')
ax.set(ylabel='Number of Baffle Spaces')
#fig.savefig('Flocculation/Images/Number_of_baffle_spaces_vs_plant_flow')
plt.show()

width_floc_channel=np.zeros(plot_points)*u.m
for i in range(plot_points):
    width_floc_channel[i]=width_floc(flow_plant[i], headloss_floc_BOD, Gt_BOD, T_BOD)
fig, ax = plt.subplots()
ax.plot(flow_plant,width_floc_channel,'-')
ax.set(xlabel='Plant Flow Rate (L/s)')
ax.set(ylabel='Floc Channel Width (m)')
plt.title('Floc channel width vs plant flow')
#fig.savefig('Flocculation/Images/Floc_channel_width_vs_plant_flow')
plt.show()